# BAIT508_Industry_Analysis
David Chen (30409156)
Wanting Xu (81971392)

## Part 1. Quantitative Analysis of the Industry Sector

In [1]:
# Package Imports and Data Loading
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Load the data of major industry sectors and their corresponding codes
dataFrameMajorGroup = pd.read_csv('data/major_groups.csv')

dataFrame10K = pd.read_csv('data/2020_10K_item1_full.csv')

dataFramePublicFirms = pd.read_csv('data/public_firms.csv')

print(dataFrameMajorGroup.head())
print(dataFramePublicFirms.head())
print(dataFrame10K.head())


   major_group                                        description
0            1                      Agricultural Production Crops
1            2  Agriculture production livestock and animal sp...
2            7                              Agricultural Services
3            8                                           Forestry
4            9                       Fishing hunting and trapping
   gvkey  fyear location      conm     ipodate   sic  prcc_c      ch      ni  \
0   1004   1994      USA  AAR CORP  1988/01/01  5080  13.375  22.487  10.463   
1   1004   1995      USA  AAR CORP  1988/01/01  5080  22.000  33.606  16.012   
2   1004   1996      USA  AAR CORP  1988/01/01  5080  30.250  51.705  23.025   
3   1004   1997      USA  AAR CORP  1988/01/01  5080  38.750  17.222  35.657   
4   1004   1998      USA  AAR CORP  1988/01/01  5080  23.875   8.250  41.671   

     asset     sale       roa  
0  425.814  451.395  0.024572  
1  437.846  504.990  0.036570  
2  529.584  589.328  0.0434

### A. Industry Sector Selection and Data Filtering

#### 1. 
List of major industry sectors and their corresponding codes has been loaded as dataframe `dataFrameMajorGroup`. Among the major industries listed we propose to look into [Industry Name]

In [2]:
selectedMajorGroupCode = 51

#### 2.

In [12]:
# Filter the data in "data/public_firms.csv" to only include the firms belonging to the industry sector(s) you have selected.
def filterByIndustry(dataFrame, majorGroupCode):
    return dataFrame[dataFrame['sic']//100 == majorGroupCode]

filteredPublicFirms = filterByIndustry(dataFramePublicFirms, selectedMajorGroupCode)

print(filteredPublicFirms)

         gvkey  fyear location                      conm ipodate   sic  \
368       1094   1995      USA                ACETO CORP     NaN  5160   
369       1094   1996      USA                ACETO CORP     NaN  5160   
370       1094   1997      USA                ACETO CORP     NaN  5160   
371       1094   1998      USA                ACETO CORP     NaN  5160   
372       1094   1999      USA                ACETO CORP     NaN  5160   
...        ...    ...      ...                       ...     ...   ...   
207716  264393   2016      USA  INNOVATIVE FOOD HOLDINGS     NaN  5140   
207717  264393   2017      USA  INNOVATIVE FOOD HOLDINGS     NaN  5140   
207718  264393   2018      USA  INNOVATIVE FOOD HOLDINGS     NaN  5140   
207719  264393   2019      USA  INNOVATIVE FOOD HOLDINGS     NaN  5140   
207720  264393   2020      USA  INNOVATIVE FOOD HOLDINGS     NaN  5140   

        prcc_c     ch     ni   asset     sale       roa  
368     16.000  1.644  7.756  86.116  164.783  0.0900

In [15]:
#How many unique firm-year ("fyear") observations are there in the filtered dataset?
numFyear =len(filteredPublicFirms.groupby('fyear'))

print(f'There are {numFyear} unique firm-year ("fyear") observations in the filtered dataset')

There are 27 unique firm-year ("fyear") observations in the filtered dataset


In [5]:
# 3. How many unique firms are there in the filtered dataset?


In [6]:
# 4. How many firms in the filtered dataset have records over all 27 years (1994-2020)?

#### 3.

### B. Preliminary Analysis


In [7]:
#To-Do:

## Part 2. Text Analysis on the Industry Sector

### C. Text Cleaning

In [8]:
#To-Do:

### D. Keyword Analysis

In [9]:
#To-Do:

### E. Word Embedding

In [10]:
#To-Do:

## Part 3. Comprehensive Analysis of One Sample Firm

### F. Firm Analysis and Strategy Suggestion

In [11]:
#To-Do: